In [16]:
import requests as req
import pandas as pd
import json
import pydash as _
# Please make sure your python version is 3.6 or lower
# https://github.com/tweepy/tweepy/issues/1017
import tweepy

## Gather

Saving udacity provided dataset into the `tw_arch`

In [17]:
tw_arch = pd.read_csv("twitter-archive-enhanced.csv")
tw_arch.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


In this part, I am going to be downloading and saving to disk `image-predictions.tsv` data from Udacity provided URL

In [2]:
# Download file
image_predictions = req.get(
    "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
).text

In [34]:
# write to disk
with open("image-predictions.tsv", "w") as f:
    f.write(image_predictions)

In [18]:
# load date to DataFrame
img_predictions = pd.read_csv("image-predictions.tsv", sep="\t")
img_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


Loading data from twitter and storing them in `tweet_json.txt`

**Please set your credentials in `credentials.json` file to download data from twitter api**

In [36]:
# Load credentials an authorize
with open('credentials.json') as f:
    creds = json.load(f)

consumer_key = creds["consumer_key"]
consumer_secret = creds["consumer_secret"]
access_token = creds["access_token"]
access_secret = creds["access_secret"]
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit_notify=True, wait_on_rate_limit=True)

In [41]:
# store tweet ids into list
tweet_ids = list(img_predictions["tweet_id"])

tweet_data = []
tweet_errors = []
# TODO: REMOVE THIS
for idx, tweet_id in enumerate(tweet_ids[:1]):
    try:
        if idx % 50 == 0:
            print("Proccessed {}/{}".format(idx, len(tweet_ids)))
        t = api.get_status(tweet_id, tweet_mode='extended')
        tweet_data.append(
            _.pick(t, ["id", "retweet_count", "favorite_count"])
        )
    except Exception as err:
        print(err)
        tweet_errors.append((tweet_id, err))
        
print("\nDone: {} downloaded out of {}".format(len(tweet_data), len(tweet_ids)))

tweets_df = pd.DataFrame(tweet_data, columns = ['id', 'retweet_count', 'favorite_count'])
# TODO: REMOVE THIS
tweets_df.to_csv('tweet_json1.txt', encoding = 'utf-8')
print("\nTweets saved to tweet_json.txt")

print("\nFollowing tweet ids have failed")
print(tweet_errors)

Proccessed 0/2075
Proccessed 50/2075
Proccessed 100/2075
Proccessed 150/2075
Proccessed 200/2075
Proccessed 250/2075
Proccessed 300/2075
Proccessed 350/2075
Proccessed 400/2075
Proccessed 450/2075
Proccessed 500/2075
Proccessed 550/2075


Rate limit reached. Sleeping for: 461


Proccessed 600/2075
Proccessed 650/2075
Proccessed 700/2075
Proccessed 750/2075
Proccessed 800/2075
Proccessed 850/2075
Proccessed 900/2075
Proccessed 950/2075
Proccessed 1000/2075
Proccessed 1050/2075
Proccessed 1100/2075
Proccessed 1150/2075
Proccessed 1200/2075
Proccessed 1250/2075
Proccessed 1300/2075
[{'code': 144, 'message': 'No status found with that ID.'}]
Proccessed 1350/2075
Proccessed 1400/2075
[{'code': 179, 'message': 'Sorry, you are not authorized to see this status.'}]
Proccessed 1450/2075


Rate limit reached. Sleeping for: 542


Proccessed 1500/2075
Proccessed 1550/2075
Proccessed 1600/2075
[{'code': 144, 'message': 'No status found with that ID.'}]
Proccessed 1650/2075
Proccessed 1700/2075
Proccessed 1750/2075
Proccessed 1800/2075
[{'code': 144, 'message': 'No status found with that ID.'}]
Proccessed 1850/2075
[{'code': 144, 'message': 'No status found with that ID.'}]
Proccessed 1900/2075
[{'code': 144, 'message': 'No status found with that ID.'}]
Proccessed 1950/2075
[{'code': 144, 'message': 'No status found with that ID.'}]
Proccessed 2000/2075
Proccessed 2050/2075
[{'code': 144, 'message': 'No status found with that ID.'}]
Done: 2067 downloaded out of 2075
Tweets saved to tweet_json.txt
Following tweet ids have failed
[(754011816964026368, TweepError([{'code': 144, 'message': 'No status found with that ID.'}])), (771004394259247104, TweepError([{'code': 179, 'message': 'Sorry, you are not authorized to see this status.'}])), (802247111496568832, TweepError([{'code': 144, 'message': 'No status found with 

In [19]:
tweets_df = pd.read_csv("tweet_json.txt")
tweets_df.head()

,Unnamed: 0,id,retweet_count,favorite_count
0,0,666020888022790149,511,2556
1,1,666029285002620928,47,130
2,2,666033412701032449,44,125
3,3,666044226329800704,141,298
4,4,666049248165822465,40,109


# Assess

### Accessing  _`twitter-archive-enhanced.csv`_ dataset

In [9]:
tw_arch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

- timestamp column should be date object
- there is lot of missing values for some columns

In [14]:
print("Duplicate count: {}".format(sum(tw_arch.duplicated())))

Duplicate count: 0


> `tw_arch` dataset seem to be duplicate free

In [21]:
tw_arch.sample(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1965,673345638550134785,NaN,NaN,2015-12-06 03:38:05 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Gerald. He's a fluffy lil yellow pup. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/673345638...,7,10,Gerald,None,None,None,None
383,827199976799354881,NaN,NaN,2017-02-02 17:00:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Charlie. He wins every game of chess h...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/827199976...,13,10,Charlie,None,None,None,None
1672,682389078323662849,NaN,NaN,2015-12-31 02:33:29 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Brody. He's a Downton Abbey Falsetto. Add...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/682389078...,9,10,Brody,None,None,None,None
2340,666099513787052032,NaN,NaN,2015-11-16 03:44:34 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Can stand on stump for what seems like a while...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666099513...,8,10,None,None,None,None,None
1474,693644216740769793,6.936422e+17,4.196984e+09,2016-01-31 03:57:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",BREAKING PUPDATE: I've just been notified that...,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
2292,667160273090932737,NaN,NaN,2015-11-19 01:59:39 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bradlay. He is a Ronaldinho Matsuyama ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667160273...,11,10,Bradlay,None,None,None,None
71,878776093423087618,NaN,NaN,2017-06-25 00:45:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Snoopy. He's a proud #PrideMonthPuppo....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/878776093...,13,10,Snoopy,None,None,None,puppo
980,749774190421639168,NaN,NaN,2016-07-04 01:17:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Lucy. She's a Benebop Cumberplop. 12/1...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/749774190...,12,10,Lucy,None,None,None,None
1490,692901601640583168,NaN,NaN,2016-01-29 02:46:29 +0000,"<a href=""http://twitter.com/download/iphone"" r...","""Fuck the system"" 10/10 https://t.co/N0OADmCnVV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/692901601...,10,10,None,None,None,None,None
1575,687476254459715584,NaN,NaN,2016-01-14 03:28:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Curtis. He's a fluffball. 11/10 would ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/687476254...,11,10,Curtis,None,None,pupper,None


- lot of missing values in columns: `in_reply_to_status_id in_reply_to_user_id retweeted_status_id retweeted_status_user_id retweeted_status_timestamp`
- We could format date to `YYYY-MM-DD HH:MM:SS`
- missing dog names in `name` column

In [43]:
# Check for names that are probably incorect
tw_arch_names_to_nan = _.filter_(tw_arch.name.unique(), lambda x: len(x) < 4 and x.islower())
print("Most likely incorrect names: {}".format(tw_arch_names_to_nan))

Most likely incorrect names: ['a', 'not', 'one', 'mad', 'an', 'my', 'his', 'all', 'old', 'the', 'by']


- This names are probably entered incorrectly or were not intended as names. Futher I could add `None` to that list

In [44]:
# Add None to list of names to be replaced
tw_arch_names_to_nan.append('None')

In [42]:
tw_arch[["doggo","floofer","pupper","puppo"]].describe()

,doggo,floofer,pupper,puppo
count,2356,2356,2356,2356
unique,2,2,2,2
top,None,None,None,None
freq,2259,2346,2099,2326


- `doggo floofer pupper puppo` columns should be boolean or melt into one column

In [78]:
tw_arch.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

- `rating_denominator` should be `10`

### Accessing *`tweet_json.txt`*  dataset

In [55]:
tweets_df.head()

,Unnamed: 0,id,retweet_count,favorite_count
0,0,666020888022790149,511,2556
1,1,666029285002620928,47,130
2,2,666033412701032449,44,125
3,3,666044226329800704,141,298
4,4,666049248165822465,40,109


- `Unnamed: 0` should be droped

In [60]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2067 entries, 0 to 2066
Data columns (total 4 columns):
Unnamed: 0        2067 non-null int64
id                2067 non-null int64
retweet_count     2067 non-null int64
favorite_count    2067 non-null int64
dtypes: int64(4)
memory usage: 64.7 KB


> Value types are ok

In [57]:
tweet_df.describe()

,Unnamed: 0,id,retweet_count,favorite_count
count,2067.000000,2.067000e+03,2067.000000,2067.000000
mean,1033.000000,7.381015e+17,2847.682148,8497.542332
std,596.835823,6.769012e+16,4926.705597,12705.248775
min,0.000000,6.660209e+17,12.000000,0.000000
25%,516.500000,6.764355e+17,607.000000,1620.500000
50%,1033.000000,7.116948e+17,1343.000000,3749.000000
75%,1549.500000,7.931430e+17,3300.000000,10660.000000
max,2066.000000,8.924206e+17,84456.000000,163674.000000


> there seem to be `0` values in `favorite_count` column

In [75]:
tweet_df.duplicated().sum()

0

> No duplicates in this dataset

### Accessing *`image-predictions.tsv`*  dataset

In [80]:
img_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [66]:
img_predictions

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


> Other than some rows are not dog related I can not see anything wrong with dataset so far

In [70]:
img_predictions.isnull().any().sum()

0

In [71]:
img_predictions.isna().any().sum()

0

In [74]:
img_predictions.duplicated().sum()

0

> No null, no Nan no duplicates. Looks good to me

### Access summary

#### Quality
- timestamp column should be date object
- incorrect timestamp format
- lot of missing values in columns: `in_reply_to_status_id in_reply_to_user_id retweeted_status_id retweeted_status_user_id retweeted_status_timestamp`
- `doggo floofer pupper puppo` columns should be boolean
- We could format date to `YYYY-MM-DD HH:MM:SS`
- missing dog names in `name` column
- Some names are probably entered incorrectly or were not intended as names. I stored them in `tw_arch_names_to_nan`
- `rating_denominator` should be `10`
- `rating_nominator` should be float datatype

#### Tideness
- remove `Unnamed: 0` column
- `doggo floofer pupper puppo`can be melt into one column
- we can merge all 3 dataset into one

## Clean